### Python Implementation of flowDBSCAN

In [19]:
# Import modules and data

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import Point

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the haversine distance between two points given their latitude and longitude.

    Parameters:
    - lat1, lon1: Latitude and longitude of the first point (in degrees)
    - lat2, lon2: Latitude and longitude of the second point (in degrees)

    Returns:
    The haversine distance in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of the Earth in kilometers (mean value)
    earth_radius = 6371.0

    # Calculate the haversine distance
    distance = earth_radius * c

    return distance

In [17]:
#Import OAs
#import OA data
wm_oas = gpd.read_file('data/west_midlands_OAs/west_midlands_OAs.shp')
wm_oas = wm_oas[wm_oas['LAD11CD'] == 'E08000026']
oa_info = pd.read_csv('data/oa_info.csv')
oa_info = oa_info.merge(wm_oas[['OA11CD']], left_on = 'oa_id', right_on = 'OA11CD', how = 'inner')
oaIndex = list(oa_info['oa_id'])
oaLatLon = oa_info[['oa_lon','oa_lat']].values

#import POI data
pois = pd.read_csv('data/POIs/pois.csv', index_col=0)

#Select local POIs
poisInRegion = []

for i,r in pois.iterrows():
    poiPoint = Point(tuple(list(r[['poi_lon','poi_lat']])))
    
    for i2, r2 in wm_oas.iterrows():
        if r2['geometry'].intersects(poiPoint):
            poisInRegion.append(r['poi_id'])

pois = pois[pois['poi_id'].isin(poisInRegion)]
poiIndex = list(pois['poi_id'])
poisLatLon = pois[['poi_lon','poi_lat']].values

In [18]:
dist_matrix = np.zeros((len(oaIndex),len(poiIndex)))

In [26]:
# Compute straight line distance matrix

oind = 0
flows = []
for o in oaLatLon:
    dind = 0
    for d in poisLatLon:
        dist_matrix[oind,dind] = haversine_distance(o[0], o[1], d[0], d[1])
        flows.append(tuple([o[0], o[1], d[0], d[1]]))
        dind += 1

    oind += 1

In [45]:
# Calculate distance matrix

# For origin and destination define x,y tuples
# For all flows in matrix define 4 dimensional tuples (xo, xd, yo, yd)

alpha = 1
beta = 1
gamma = 1

i = 0
j = 1

flow_i = flows[i]
flow_j = flows[j]
flow_dist_i = dist_matrix[0,0]
flow_dist_j = dist_matrix[0,1]

dist = (alpha*((flow_i[0] - flow_j[0]) ** 2) + beta*(((flow_i[2] - flow_j[2]) ** 2) + ((flow_i[3] - flow_j[3]) ** 2))) / ((flow_dist_i * flow_dist_j)**gamma)

print(dist)

4.746558276908352e-05


In [44]:
flow_dist_j

9.51627805652647

In [37]:
((flow_i[0] - flow_j[0]) ** 2) + ((flow_i[1] - flow_j[1]) ** 2)

0.0

0.002303648416997166

In [38]:
print(flow_i)
print(flow_j)

(-1.530235434, 52.43459291, -1.491767645, 52.40960693)
(-1.530235434, 52.43459291, -1.445519686, 52.42244339)


In [ ]:
# Calculate CoreD

In [ ]:
# Calculate MinReachD matrix

In [ ]:
# Sort MST to get dendogram

In [ ]:
# Simplify Dendogramn based on MinFlow

In [ ]:
# Extract flow clusters